In [1]:
import requests, json
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import sys
import time
import requests, json
import numpy as np
import calendar
from datetime import datetime
import random
import glob

In [2]:
#converts numpy types to python types, otherwise json conversion produces an error. call json.dumps(***, cls=MyEncoder)
class MyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(MyEncoder, self).default(obj)


# returns strings in the conventional format for iot.ids
def idstr(idinput):
    return(str(idinput).lower().replace(" ", "_").replace("/", "_").replace("ä", "ae").replace("ö", "oe").replace("ü", "ue").replace("ß", "ss"))


# returns the first 7 digits of the sha1 hash of the input string
def hashfunc(inputstring, printme):
    returnhash= hashlib.sha1(bytes(str(inputstring), 'utf-8')).hexdigest()[0:7]
    if printme == True:
        print("Converting '" + str(inputstring) + "' to hash '" + str(returnhash) +"'")
    return(returnhash)

#url="https://smartaqnet-dev.dmz.teco.edu/v1.0"
url="https://api.smartaq.net/v1.0/"

In [59]:
#requests.delete("http://smartaqnet-dev.teco.edu:8080/FROST-Server/v1.0/Things")
#requests.delete("http://smartaqnet-dev.teco.edu:8080/FROST-Server/v1.0/Sensors")
#requests.delete("http://smartaqnet-dev.teco.edu:8080/FROST-Server/v1.0/Locations")
#requests.delete("http://smartaqnet-dev.teco.edu:8080/FROST-Server/v1.0/ObservedProperties")
#requests.delete("http://smartaqnet-dev.teco.edu:8080/FROST-Server/v1.0/FeaturesOfInterest")
#requests.delete("http://smartaqnet-dev.teco.edu:8080/FROST-Server/v1.0/Datastreams")
#requests.delete("http://smartaqnet-dev.teco.edu:8080/FROST-Server/v1.0/Observations")

In [60]:
json.loads(requests.get(url + "/Things?$filter=properties/operator_url%20eq%20%27lfu.bayern.de%27&$select=@iot.id").text)

{'value': [{'@iot.id': 'saqn:t:87704c8'},
  {'@iot.id': 'saqn:t:12b713d'},
  {'@iot.id': 'saqn:t:b666034'},
  {'@iot.id': 'saqn:t:4049564'},
  {'@iot.id': 'saqn:t:d42cbb8'},
  {'@iot.id': 'saqn:t:504a3c6'}]}

## Patching Properties

In [82]:
# Things, Datasteams, Locations, ...
category_to_patch = "/Things"

#saqn:...
entity_to_patch_id = 'saqn:t:504a3c6'

#optional get request for template on what to patch
query = ""

loaded_data = json.loads(requests.get(url + category_to_patch + "('" + entity_to_patch_id + "')" + query).text)
loaded_data

In [84]:
# # loaded_data["properties"]["station_code"]
# foafagent = {
#       "@id": "https://www.umweltbundesamt.de/impressum",
#       "@type": "foaf:Organization",
#       "foaf:name": "Umweltbundesamt"
#     }
# loaded_data["properties"]["license"] = {"dcatde:maintainer": "Umweltbundesamt", "foaf:agent" : foafagent}
# loaded_data["properties"]
loaded_data["properties"]["license"] = {}
loaded_data["properties"]

{'dem_status': 'M',
 'station_code': 'DEBY008',
 'station_local_code': 'DEBY008',
 'station_name': 'Augsburg/Haunstetten',
 'station_start_date': 19750101,
 'station_end_date': 20010331.0,
 'station_latitude_d': 48.308269,
 'station_latitude_dms': '+048.18.29',
 'station_longitude_d': 10.907772999999999,
 'station_longitude_dms': '+010.54.27',
 'station_altitude': 502.0,
 'type_of_station': 'background',
 'station_type_of_area': 'suburban',
 'station_ozone_classification': 'suburban',
 'station_subcat_rural_background': 'unknown',
 'network_code': 'DE007A',
 'operator_url': 'lfu.bayern.de',
 'license': {}}

In [85]:
#what to patch?

patch_what ={"properties": loaded_data["properties"]}


#send patch request
resp_dev=requests.patch(url + category_to_patch + "('" + entity_to_patch_id + "')",json=patch_what)
# resp=requests.patch("http://smartaqnet.teco.edu/v1.0/" + category_to_patch + "('" + entity_to_patch_id + "')",json=patch_what)

print(resp_dev)
# print(resp)

<Response [200]>


## Patching Locations for Workshop Nodes

In [8]:
#nr=11
#a_location=json.loads(requests.get("http://smartaqnet.teco.edu/v1.0/Locations?$select=@iot.id,location").text)["value"][nr]
#a_location

a_location=json.loads(requests.get("https://smartaqnet.teco.edu/v1.0/Locations('geo%3A10.94%2C48.38%2Cnan')?$select=@iot.id,location").text)
a_location


#breit ist das große, lang das kleine
#im json muss das kleine zuerst: [lang, breit]

{'location': {'type': 'Point', 'coordinates': [48.37616, 10.94464]},
 '@iot.id': 'geo:10.94,48.38,nan'}

In [5]:
breit=a_location["location"]["coordinates"][0]
lang=a_location["location"]["coordinates"][1]

In [6]:
requests.patch("http://smartaqnet.teco.edu/v1.0/Locations('" + a_location["@iot.id"] + "')",json={"location": {"type": "Point", "coordinates": [lang,breit]}})

<Response [403]>

In [7]:
json.loads(requests.get("http://smartaqnet.teco.edu/v1.0/Locations('" + str(a_location["@iot.id"]) + "')").text)

{'name': 'WÃ¶rnitzstrasse 16, 86165 Augsburg ',
 'description': '',
 'encodingType': 'application/vnd.geo+json',
 'location': {'type': 'Point', 'coordinates': [48.37616, 10.94464]},
 'HistoricalLocations@iot.navigationLink': "https://smartaqnet.teco.edu/v1.0/Locations('geo%3A10.94%2C48.38%2Cnan')/HistoricalLocations",
 'Things@iot.navigationLink': "https://smartaqnet.teco.edu/v1.0/Locations('geo%3A10.94%2C48.38%2Cnan')/Things",
 '@iot.id': 'geo:10.94,48.38,nan',
 '@iot.selfLink': "https://smartaqnet.teco.edu/v1.0/Locations('geo%3A10.94%2C48.38%2Cnan')"}

In [38]:
testthing = {'name': 'Testthing',
            'description': 'thing for testing',
             'properties': 
                 {
                 'operator_url': 'www.smartaq.net', 
                 'license': {'03-2019': {'type': 'CCO', 
                                        'owner': 'schorsch',
                                        'link': 'www.cc0.de'},
                             '05-2019': {'type': 'CC1',
                                         'owner': 'schorschsfrau',
                                         'link': 'www.cc1.de'}
                            },
                'firmware_no': {'03-2019': 'v0.8',
                                '06-2019': 'v0.9'},
                 }
            }

In [39]:
requests.patch("http://scratchpad.sensorup.com/OGCSensorThings/v1.0/Things(2666026)", json=testthing)

<Response [200]>

In [11]:
licenseupdate = 

In [12]:
requests.post("https://smartaqnet-dev.dmz.teco.edu/v1.0/Things('saqn%3At%3Abuggabugga')/properties/license", json=firmwareupdate)

<Response [400]>

In [15]:
firmwareupdate = {'09-2019': 'v1.0'}

In [25]:
requests.patch("https://smartaqnet-dev.dmz.teco.edu/v1.0/Things('saqn%3At%3Abuggabugga')/properties/firmware_no", json=firmwareupdate)

<Response [403]>

In [28]:
testurl = "https://scratchpad.sensorup.com/OGCSensorThings/v1.0/Things"

In [23]:
props  = json.loads(requests.get("https://smartaqnet-dev.dmz.teco.edu/v1.0/Things('saqn%3At%3A87704c8')/properties").text)

In [24]:
requests.patch("https://smartaqnet-dev.dmz.teco.edu/v1.0/Things('saqn%3At%3A87704c8')", json=props)

<Response [200]>

In [25]:
props  = json.loads(requests.get("https://api.smartaq.net/v1.0/Things('saqn%3At%3A87704c8')/properties").text)

In [26]:
requests.patch("https://api.smartaq.net/v1.0/Things('saqn%3At%3A87704c8')", json=props)

<Response [403]>

In [31]:
dem={"dem_status": "F"}

In [32]:
dem

{'dem_status': 'F'}

In [36]:
requests.patch("https://smartaqnet-dev.dmz.teco.edu/v1.0/Things('saqn%3At%3A87704c8')/properties", json=dem)

<Response [400]>

# Creating Historical Locations

In [17]:
# Things, Datasteams, Locations, ...
category_to_patch = "/Things"

#saqn:...
entity_to_patch_id = 'saqn:t:504a3c6'

#optional get request for template on what to patch
query = "?$expand=HistoricalLocations,Locations&$select=HistoricalLocations,Locations"

loaded_data = json.loads(requests.get(url + category_to_patch + "('" + entity_to_patch_id + "')" + query).text)
loaded_data

{'Locations@iot.navigationLink': "https://api.smartaq.net/v1.0/Things('saqn%3At%3A504a3c6')/Locations",
 'Locations': [{'name': 'Location at latitude 48.308269 and longitude 10.907772999999999',
   'description': 'located at Augsburg/Haunstetten',
   'encodingType': 'application/vnd.geo+json',
   'location': {'type': 'Point', 'coordinates': [10.907773, 48.308269, 502.0]},
   '@iot.id': 'geo:48.308269,10.907772999999999,502.0',
   '@iot.selfLink': "https://api.smartaq.net/v1.0/Locations('geo%3A48.308269%2C10.907772999999999%2C502.0')"}],
 'HistoricalLocations@iot.navigationLink': "https://api.smartaq.net/v1.0/Things('saqn%3At%3A504a3c6')/HistoricalLocations",
 'HistoricalLocations': [{'time': '2019-03-19T18:28:11.582Z',
   '@iot.id': 'c0620682-4a74-11e9-a32c-2b8e026b3864',
   '@iot.selfLink': "https://api.smartaq.net/v1.0/HistoricalLocations('c0620682-4a74-11e9-a32c-2b8e026b3864')"}]}

In [21]:
loaded_data["HistoricalLocations"]

[{'time': '2019-03-19T18:28:11.582Z',
  '@iot.id': 'c0620682-4a74-11e9-a32c-2b8e026b3864',
  '@iot.selfLink': "https://api.smartaq.net/v1.0/HistoricalLocations('c0620682-4a74-11e9-a32c-2b8e026b3864')"}]

In [22]:
test_hist_loc = {'time': '2019-03-19T18:28:15.000Z'}

In [23]:
requests.post("https://smartaqnet-dev.dmz.teco.edu/v1.0/HistoricalLocations", json=json.dumps(test_hist_loc))

<Response [400]>

In [26]:
hist_loc_new = json.loads(requests.get("https://smartaqnet-dev.dmz.teco.edu/v1.0/HistoricalLocations('4d333836-45c2-11e9-b79c-8fb22b87d068')").text)

In [27]:
hist_loc[0]["time"]

'2019-03-13T19:00:00.000Z'

In [36]:
hist_loc = json.loads(requests.get("https://smartaqnet-dev.dmz.teco.edu/v1.0/HistoricalLocations('4d333836-45c2-11e9-b79c-8fb22b87d068')").text)

In [37]:
print(hist_loc["time"])

'2019-03-13T19:00:00.000Z'

# Patching Historical Locations

In [1]:
import requests, json

#example historical location
hist_loc_id = '4d333836-45c2-11e9-b79c-8fb22b87d068'

#the original time
old_hist_loc_time = {"time": str(json.loads(requests.get("https://smartaqnet-dev.dmz.teco.edu/v1.0/HistoricalLocations('" + hist_loc_id + "')").text)["time"])}
print(old_hist_loc_time["time"])

#define and patch new time and check server if it accepted the new time
new_hist_loc_time = {"time": '2019-03-13T19:00:00.000Z'}
requests.patch("https://smartaqnet-dev.dmz.teco.edu/v1.0/HistoricalLocations('" + hist_loc_id + "')", json=new_hist_loc_time)
print(json.loads(requests.get("https://smartaqnet-dev.dmz.teco.edu/v1.0/HistoricalLocations('" + hist_loc_id + "')").text)["time"])

#patch old time and check if server accepted
requests.patch("https://smartaqnet-dev.dmz.teco.edu/v1.0/HistoricalLocations('" + hist_loc_id + "')", json=old_hist_loc_time)
print(json.loads(requests.get("https://smartaqnet-dev.dmz.teco.edu/v1.0/HistoricalLocations('" + hist_loc_id + "')").text)["time"])

2019-03-13T19:00:43.268Z
2019-03-13T19:00:00.000Z
2019-03-13T19:00:43.268Z
